# **Cantilever Beam:**

**Boundary Conditions:**   ```v[x = 0] = 0; dv/dx[x = 0] = 0```

---
Point Load
---

In [ ]:
# Newton-Raphson Solver for Nonlinear Beam Deflection
# ---------------------------------------------------
# This script solves a nonlinear boundary value problem of beam deflection under a point load
# using the Newton-Raphson method. The nonlinearity arises from large deformation effects.

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

def newton_raphson_system(N, init_guess='uniform', dv_dx_at_firstPoint='FD2', tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear beam deflection equation using the Newton-Raphson method.

    Parameters:
        N (int): Number of grid intervals (minimum 3, max ~350 for stability).
        init_guess (str): 'random', 'uniform', or 'zeros' - initial guess for deflection.
        dv_dx_at_firstPoint (str): Finite difference method at x=0 ('FD2', 'FD1', 'Direct').
        tol (float): Convergence tolerance for Newton-Raphson.
        max_iter (int): Maximum allowed iterations.
        FD1: Finite Difference Method 1
        FD2: Finite Difference Method 2

    Returns:
        x (array): Discretized x values.
        v (array): Computed beam deflection at each x.
        L (float): Beam length.
        P (float): Point load applied at free end.
        EI (float): Flexural rigidity.
    """

    # Valid input checks
    valid_init_guesses = ['random', 'uniform', 'zeros']
    valid_dv_dx_methods = ['FD2', 'FD1', 'Direct']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")
    if dv_dx_at_firstPoint not in valid_dv_dx_methods:
        raise ValueError(f"Invalid dv/dx method '{dv_dx_at_firstPoint}'. Must be one of {valid_dv_dx_methods}.")

    # Problem parameters
    L = 1.0            # Beam length
    dx = L / N         # Grid spacing
    x = np.linspace(0, L, N+1)  # Discretized domain
    EI = 2000          # Flexural rigidity
    P  = -100.0        # Point load at free end (negative: downward)
    w  = 0.5           # Relaxation factor for update step

    # Initialize deflection v(x) based on guess
    if init_guess == 'random':
        v = np.random.uniform(0, 0.01 * np.sign(P), N+1)
        v[0] = 0  # Apply boundary condition
    elif init_guess == 'uniform':
        v = 0.0001 * np.sign(P) * x[:]
        v[0] = 0
    elif init_guess == 'zeros':
        v = np.zeros(N+1)
        v[0] = 0

    # Bending moment along the beam
    M = np.zeros(N+1)
    for i in range(N+1):
        M[i] = -P * (L - x[i])  # Linearly varying moment due to point load at x=L

    # Residual function for Newton-Raphson
    def residual(v, M):
        R = np.zeros(N+1)
        v_full = np.copy(v)
        v_full[0] = 0  # Apply BC: v(0) = 0

        for i in range(N+1):
            if i == 0:
                # Apply slope condition dv/dx at x=0
                if dv_dx_at_firstPoint == 'FD2':
                    R[i] = (-3*v_full[i] + 4*v_full[i+1] - v_full[i+2]) / (2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    R[i] = (v_full[i+1] - v_full[i]) / dx
                elif dv_dx_at_firstPoint == 'Direct':
                    R[i] = (2*v_full[i] - 5*v_full[i+1] + 4*v_full[i+2] - v_full[i+3]) / dx**2 + M[i]/EI

            elif i == N:
                # At the free end, apply nonlinear moment-curvature relation
                delta = 3*v_full[i] - 4*v_full[i-1] + v_full[i-2]
                R[i] = (2*v_full[i] - 5*v_full[i-1] + 4*v_full[i-2] - v_full[i-3]) / dx**2 \
                       + (M[i]/EI) * (1 + (delta**2) / (4*dx**2))**(3/2)
            else:
                # Interior points
                delta = v_full[i+1] - v_full[i-1]
                R[i] = (v_full[i+1] - 2*v_full[i] + v_full[i-1]) / dx**2 \
                       + (M[i]/EI) * (1 + (delta**2) / (4*dx**2))**(3/2)
        return R

    # Jacobian matrix for Newton-Raphson
    def jacobian(v, M):
        J = np.zeros((N+1, N+1))
        v_full = np.copy(v)
        v_full[0] = 0

        for i in range(N+1):
            if i == 0:
                if dv_dx_at_firstPoint == 'FD2':
                    J[i, i+1] = 4 / (2*dx)
                    J[i, i+2] = -1 / (2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    J[i, i+1] = 1 / dx
                elif dv_dx_at_firstPoint == 'Direct':
                    J[i, i+1] = -5 / dx**2
                    J[i, i+2] = 4 / dx**2
                    J[i, i+3] = -1 / dx**2

            elif i == N:
                delta = 3*v_full[i] - 4*v_full[i-1] + v_full[i-2]
                d_common = (M[i]/EI) * (3/2) * (1 + delta**2 / (4*dx**2))**(1/2) * 1/(4*dx**2) * 2*delta
                J[i, i-3] = -1 / dx**2
                J[i, i-2] = 4 / dx**2 + d_common * 1
                J[i, i-1] = -5 / dx**2 + d_common * (-4)
                J[i, i]   = 2 / dx**2 + d_common * 3
            else:
                delta = v_full[i+1] - v_full[i-1]
                d_common = (M[i]/EI) * (3/2) * (1 + delta**2 / (4*dx**2))**(1/2) * 1/(4*dx**2) * 2
                J[i, i-1] = 1 / dx**2 + d_common * (v_full[i-1] - v_full[i+1]) if i > 1 else 0
                J[i, i]   = -2 / dx**2
                J[i, i+1] = 1 / dx**2 + d_common * (v_full[i+1] - v_full[i-1])
        return J

    # Newton-Raphson Iteration Loop
    for iteration in range(max_iter):
        R = residual(v, M)
        print(np.linalg.norm(R))  # Track convergence
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, M)
        try:
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]

        v += w * delta_v  # Relaxation update

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    v[0] = 0  # Ensure boundary is enforced
    return x, v, L, P, EI


# === Run the Solver ===

# Inputs from user
'''
    Minimum N = 3
    Recommended N <= 350 to avoid Jacobian singularity.
'''
N = 250
init_guess = input("Enter initial guess type (random, uniform, zeros): ").strip().lower()
dv_dx_at_firstPoint = input("Enter dv/dx at x = 0 (FD2, FD1, Direct): ").strip().upper()

# Compute solution
x, v, L, P, EI = newton_raphson_system(N, init_guess, dv_dx_at_firstPoint)

# Print final solution
print("\nDeflection Matrix:\n", v)

# Compare with linear Euler-Bernoulli beam theory
df = pd.DataFrame({
    'x': x,
    'Newton-Raphson Solution (point load)': v,
    'Euler-Bernoulli Solution (point load)': P * x**2 / (6 * EI) * (3 * L - x)
})

# Interactive Plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['x'], y=df['Newton-Raphson Solution (point load)'], mode='lines', name='Newton-Raphson'))
fig.add_trace(go.Scatter(x=df['x'], y=df['Euler-Bernoulli Solution (point load)'], mode='lines', name='Euler-Bernoulli'))

fig.update_layout(
    title="Deflection Profile of Beam (Nonlinear vs Linear)",
    xaxis_title="x (position along beam)",
    yaxis_title="Deflection v(x)",
    template="plotly_white"
)

fig.show()


---
Point load(With Table of Error) for some EI value
---

In [ ]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import pandas as pd

# Newton-Raphson Solver for nonlinear beam deflection
def newton_raphson_system(N, EI, tol=1e-6, max_iter=1000):
    L = 1.0                      # Length of the beam (in meters)
    dx = L / N                  # Step size
    x = np.linspace(0, L, N + 1)  # Discretized x-coordinates
    K = 1 / EI                  # Flexibility (inverse of stiffness)
    P = -100.0                  # Point load at the free end (N)
    w = 0.5                     # Relaxation factor to stabilize convergence
    inv_dx = 1 / dx
    inv_dx2 = inv_dx ** 2


    #Here I use FD2 and uniform initial guess.

    # Initial guess for deflection profile (small displacement in direction of load)
    v = 0.0001 * np.sign(P) * x[:]
    v[0] = 0  # Fixed end boundary condition

    # Bending moment distribution due to point load at the tip
    M = -P * (L - x)

    # Residual function for the system of nonlinear equations
    def residual(v, M):
        R = np.zeros(N + 1)
        v_full = np.zeros(N + 1)
        v_full[:] = v
        v_full[0] = 0  # Fixed boundary condition

        for i in range(N + 1):
            if i == 0:
                # Forward difference for slope at x = 0
                R[i] = ((-3 * v_full[i] + 4 * v_full[i + 1] - v_full[i + 2]) * inv_dx) * 0.5
            elif i == N:
                # Backward difference at free end (nonlinear term included)
                dv = 3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]
                R[i] = (2 * v_full[i] - 5 * v_full[i - 1] + 4 * v_full[i - 2] - v_full[i - 3]) * inv_dx2 \
                       + (M[i] * K) * (1 + (inv_dx2 / 4) * dv**2) ** (3 / 2)
            else:
                # Central difference with nonlinearity
                dv = v_full[i + 1] - v_full[i - 1]
                R[i] = (v_full[i + 1] - 2 * v_full[i] + v_full[i - 1]) * inv_dx2 \
                       + (M[i] * K) * (1 + (inv_dx2 / 4) * dv**2) ** (3 / 2)
        return R

    # Jacobian matrix of the residuals (used in Newton-Raphson)
    def jacobian(v, M):
        J = np.zeros((N + 1, N + 1))
        v_full = np.zeros(N + 1)
        v_full[:] = v
        v_full[0] = 0

        for i in range(N + 1):
            if i == 0:
                # Derivative of forward difference at the start
                J[i, i], J[i, i + 1], J[i, i + 2] = 0, 2 * inv_dx, -0.5 * inv_dx
            elif i == N:
                # Derivative of residual at the free end
                dv = 3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]
                common = (M[i] * K) * (3 / 2) * (1 + (inv_dx2 / 4) * dv**2) ** 0.5 * (inv_dx2 / 4) * 2 * dv
                J[i, i - 3] = -inv_dx2
                J[i, i - 2] = 4 * inv_dx2 + common * 1
                J[i, i - 1] = -5 * inv_dx2 + common * (-4)
                J[i, i] = 2 * inv_dx2 + common * 3
            else:
                # Central difference + nonlinearity
                dv = v_full[i + 1] - v_full[i - 1]
                common = (M[i] * K) * (3 / 2) * (1 + (inv_dx2 / 4) * dv**2) ** 0.5 * (inv_dx2 / 4)
                J[i, i - 1] = 0 if i == 1 else inv_dx2 + common * (-2 * dv)
                J[i, i] = -2 * inv_dx2
                J[i, i + 1] = inv_dx2 + common * (2 * dv)
        return J

    # Main Newton-Raphson iteration loop
    for iteration in range(max_iter):
        R = residual(v, M)
        if np.linalg.norm(R) < tol:
            print(f"Converged in {iteration+1} iterations.")
            break

        J = jacobian(v, M)
        try:
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            print(f"Jacobian is singular at iteration {iteration+1}. Using pseudo-inverse.")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]

        v += w * delta_v  # Update the solution with relaxation

        if np.linalg.norm(delta_v) < tol:
            print(f"Converged in {iteration+1} iterations.")
            break
    else:
        raise ValueError("Newton-Raphson did not converge within max iterations.")

    v_full = np.zeros(N + 1)
    v_full[:] = v
    v_full[0] = 0  # Reinforce boundary condition

    return x, v_full, L, P, K, inv_dx, inv_dx2

# Analytical solution using Euler-Bernoulli beam theory
def euler_bernoulli_deflection(x, L, P, EI):
    return (P * x**2 * (3 * L - x)) / (6 * EI)

# Simulation Parameters
L = 1.0          # Length of beam
N = 150          # Number of elements
x = np.linspace(0, L, N + 1)
P = -100.0       # Load at free end
EI_arr = [20, 30, 40, 50, 100, 500, 1000, 2000]  # Flexural rigidity values

table = []  # To store results for each EI

# Run simulation for each EI and collect results
for EI in EI_arr:
    print(f"\nSolving for EI = {EI}")
    x, v, L, P, K, inv_dx, inv_dx2 = newton_raphson_system(N, EI)
    v_euler = euler_bernoulli_deflection(x, L, P, EI)
    v_euler_at_tip = float(v_euler[-1])
    v_NR_at_tip = float(v[-1])
    final_error = ((v_NR_at_tip - v_euler_at_tip) * 100) / v_euler_at_tip
    table.append([EI, v_euler_at_tip, v_NR_at_tip, final_error])

# Display results in a formatted table
df = pd.DataFrame(table, columns=["EI", "v_euler_tip", "v_NR_tip", "Difference (%)"])
print("\nResults Table:\n")
print(df.to_string(index=False))

# Plot 1: Error % vs EI (log scale)
plt.figure(figsize=(8, 5))
plt.plot(df["EI"], df["Difference (%)"], marker='o', linestyle='-', color='r', label="Difference (%)")
plt.xlabel("Flexural Rigidity (EI)")
plt.ylabel("Relative Percentage Difference (%)")
plt.title("Difference with EI Variation (At the Free End)")
plt.xscale("log")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.minorticks_on()
plt.tick_params(axis="both", which="both", direction="in")
ax = plt.gca()
major_ticks = ax.yaxis.get_majorticklocs()
if len(major_ticks) > 1:
    major_step = np.diff(major_ticks).mean()
    ax.yaxis.set_minor_locator(MultipleLocator(major_step / 3))
plt.legend()
plt.show()

# Plot 2: Deflection values vs EI
plt.figure(figsize=(8, 5))
plt.plot(df["EI"], df["v_euler_tip"], marker='s', linestyle='-', label="v_euler_tip")
plt.plot(df["EI"], df["v_NR_tip"], marker='^', linestyle='-', label="v_NR_tip")
plt.xlabel("Flexural Rigidity (EI)")
plt.ylabel("Deflection at the Free End")
plt.title("Deflection at the Free End vs EI")
plt.xscale("log")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.minorticks_on()
plt.tick_params(axis="both", which="both", direction="in")
ax = plt.gca()
major_ticks = ax.yaxis.get_majorticklocs()
if len(major_ticks) > 1:
    major_step = np.diff(major_ticks).mean()
    ax.yaxis.set_minor_locator(MultipleLocator(major_step / 3))
plt.legend(loc="lower right")
plt.show()


---
Comparison of Point Load v/s Distributed Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess, dv_dx_at_firstPoint, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method for beam deflection.

    Parameters:
        N (int): Number of grid spaces.
        init_guess (str): Type of initial guess for the deflection ('random', 'uniform', 'zeros').
        dv_dx_at_firstPoint (str): Finite difference method for the boundary condition at x=0 ('FD2', 'FD1', 'Direct').
        tol (float, optional): Tolerance for convergence. Defaults to 1e-6.
        max_iter (int, optional): Maximum number of iterations. Defaults to 1000.
        FD1: Finite Difference Method 1
        FD2: Finite Difference Method 2

    Returns:
        x (np.array): Discretized x values along the beam.
        v_full_point (np.array): Solution for deflection v(x) under point load.
        v_full_cont (np.array): Solution for deflection v(x) under continuous load.
        L (float): Length of the beam.
        P (float): Point load at the free end.
        W (float): Continuous load.
        EI (float): Flexural rigidity of the beam.
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']
    valid_dv_dx_methods = ['FD2', 'FD1', 'Direct']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")
    if dv_dx_at_firstPoint not in valid_dv_dx_methods:
        raise ValueError(f"Invalid dv/dx method '{dv_dx_at_firstPoint}'. Must be one of {valid_dv_dx_methods}.")

    # Discretization and Material Properties
    L = 1.0           # Length of the beam (m)
    dx = L / N        # Grid spacing
    x = np.linspace(0, L, N + 1)  # Discretized x values
    EI = 2000         # Flexural rigidity (N·m^2)
    P = -100.0        # Point load at the free end (N) - Negative for downward
    W = -100.0        # Continuous Load (N/m) - Negative for downward
    w = 0.5           # Relaxation factor for Newton-Raphson update

    # Initial guess for continuous load
    if init_guess == 'random':
        v_cont = np.random.uniform(0, 0.01 * np.sign(W), N + 1)  # Random initial guess, scaled by load sign
        v_cont[0] = 0  # Apply boundary condition v(0) = 0
    elif init_guess == 'uniform':
        v_cont = 0.0001 * np.sign(W) * x[:]  # Uniform initial guess, scaled by load sign
        v_cont[0] = 0  # Apply boundary condition v(0) = 0
    elif init_guess == 'zeros':
        v_cont = np.zeros(N + 1)  # Zero initial guess
        v_cont[0] = 0  # Apply boundary condition v(0) = 0

    # Initial guess for point load
    if init_guess == 'random':
        v_point = np.random.uniform(0, 0.01 * np.sign(P), N + 1) # Random initial guess, scaled by load sign
        v_point[0] = 0 # Apply boundary condition v(0) = 0
    elif init_guess == 'uniform':
        v_point = 0.0001 * np.sign(P) * x[:]  # Uniform initial guess, scaled by load sign
        v_point[0] = 0  # Apply boundary condition v(0) = 0
    elif init_guess == 'zeros':
        v_point = np.zeros(N + 1)  # Zero initial guess
        v_point[0] = 0  # Apply boundary condition v(0) = 0

    # Moment Equation - Analytical expressions for bending moment
    M_cont = np.zeros(N + 1)
    M_point = np.zeros(N + 1)
    for i in range(N + 1):
        # Bending moment for a cantilever beam with continuous load W
        M_cont[i] = W * L * x[i] - (W * (x[i] ** 2)) / 2 - (W * (L ** 2)) / 2
        # Bending moment for a cantilever beam with point load P at the free end
        M_point[i] = -P * (L - x[i])

    # Residual Function
    def residual(v, M):
        """
        Compute the residual vector R(v).
        This represents the error in satisfying the beam deflection equation.
        """
        R = np.zeros(N + 1)
        v_full = np.zeros(N + 1)
        v_full[:] = v
        v_full[0] = 0  # Enforce boundary condition v(0) = 0

        for i in range(0, N + 1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0 (zero slope at fixed end)
                if dv_dx_at_firstPoint == 'FD2':
                    # Second-order central finite difference for derivative
                    R[i] = (-3 * v_full[i] + 4 * v_full[i + 1] - v_full[i + 2]) / (2 * dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    # First-order forward finite difference for derivative
                    R[i] = (v_full[i + 1] - v_full[i]) / dx
                elif dv_dx_at_firstPoint == 'Direct':
                    # Direct application of the beam equation at the boundary (requires careful formulation)
                    # This term seems to be a mix of second derivative and moment, needs verification for a boundary condition
                    R[i] = (2 * v_full[i] - 5 * v_full[i + 1] + 4 * v_full[i + 2] - v_full[i + 3]) / dx ** 2 + (M[i] / EI)
            elif i == N:
                # Beam deflection equation at the free end (i=N)
                # This uses a backward finite difference for the second derivative and includes the non-linear term
                R[i] = (2 * v_full[i] - 5 * v_full[i - 1] + 4 * v_full[i - 2] - v_full[i - 3]) / dx ** 2 + \
                       (M[i] / EI) * (1 + 1 / (4 * dx ** 2) * (3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]) ** 2) ** (3 / 2)
            else:
                # Beam deflection equation for interior points
                # This uses central finite difference for the second derivative and includes the non-linear term
                R[i] = (v_full[i + 1] - 2 * v_full[i] + v_full[i - 1]) / dx ** 2 + \
                       (M[i] / EI) * (1 + 1 / (4 * dx ** 2) * (v_full[i + 1] - v_full[i - 1]) ** 2) ** (3 / 2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """
        Compute the Jacobian matrix J(v).
        The Jacobian contains the partial derivatives of the residual functions with respect to each v_i.
        """
        J = np.zeros((N + 1, N + 1))
        v_full = np.zeros(N + 1)
        v_full[:] = v
        v_full[0] = 0  # Enforce boundary condition v(0) = 0

        for i in range(0, N + 1):
            if i == 0:
                # Derivatives for the boundary condition at x=0
                if dv_dx_at_firstPoint == 'FD2':
                    J[i, i], J[i, i + 1], J[i, i + 2] = 0, 4 / (2 * dx), -1 / (2 * dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    J[i, i], J[i, i + 1] = 0, 1 / dx
                elif dv_dx_at_firstPoint == 'Direct':
                    J[i, i], J[i, i + 1], J[i, i + 2], J[i, i + 3] = 0, -5 / dx ** 2, 4 / dx ** 2, -1 / dx ** 2

            elif i == N:
                # Derivatives for the beam equation at the free end (i=N)
                # These are derived from the partial derivatives of the residual R[N]
                J[i, i - 3] = -1 / dx ** 2
                J[i, i - 2] = 4 / dx ** 2 + (M[i] / EI) * (3 / 2) * \
                              (1 + 1 / (4 * dx ** 2) * (3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]) ** 2) ** (1 / 2) * \
                              1 / (4 * dx ** 2) * 2 * (3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]) * (1)
                J[i, i - 1] = -5 / dx ** 2 + (M[i] / EI) * (3 / 2) * \
                              (1 + 1 / (4 * dx ** 2) * (3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]) ** 2) ** (1 / 2) * \
                              1 / (4 * dx ** 2) * 2 * (3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]) * (-4)
                J[i, i] = 2 / dx ** 2 + (M[i] / EI) * (3 / 2) * \
                          (1 + 1 / (4 * dx ** 2) * (3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]) ** 2) ** (1 / 2) * \
                          1 / (4 * dx ** 2) * 2 * (3 * v_full[i] - 4 * v_full[i - 1] + v_full[i - 2]) * (3)
            else:
                # Derivatives for the beam equation at interior points
                # These are derived from the partial derivatives of the residual R[i]
                if i == 1:
                    J[i, i - 1] = 0 # v[0] is fixed to 0, so its derivative is 0
                else:
                    J[i, i - 1] = 1 / dx ** 2 + (M[i] / EI) * (3 / 2) * \
                                  (1 + 1 / (4 * dx ** 2) * (v_full[i + 1] - v_full[i - 1]) ** 2) ** (1 / 2) * \
                                  1 / (4 * dx ** 2) * (2 * v_full[i - 1] - 2 * v_full[i + 1])
                J[i, i] = -2 / dx ** 2
                J[i, i + 1] = 1 / dx ** 2 + (M[i] / EI) * (3 / 2) * \
                              (1 + 1 / (4 * dx ** 2) * (v_full[i + 1] - v_full[i - 1]) ** 2) ** (1 / 2) * \
                              1 / (4 * dx ** 2) * (2 * v_full[i + 1] - 2 * v_full[i - 1])
        return J

    # Newton-Raphson iteration for continuous load
    print("\n--- Solving for Continuous Load ---")
    for iteration in range(max_iter):
        R_cont = residual(v_cont, M_cont)
        print(f"Iteration {iteration + 1}: Residual Norm = {np.linalg.norm(R_cont):.4e}")
        if np.linalg.norm(R_cont) < tol:
            print(f"\nConverged in {iteration + 1} iterations.\n")
            break

        J_cont = jacobian(v_cont, M_cont)
        try:
            # Solve the linear system J * delta_v = -R
            delta_v_cont = np.linalg.solve(J_cont, -R_cont)
        except np.linalg.LinAlgError:
            # If Jacobian is singular, use pseudo-inverse for a least-squares solution
            print(f"\nJacobian is singular for iteration {iteration + 1}. Using pseudo-inverse.")
            delta_v_cont = np.linalg.lstsq(J_cont, -R_cont, rcond=None)[0]  # Uses pseudo-inverse

        v_cont += w * delta_v_cont  # Update solution with relaxation factor

        if np.linalg.norm(delta_v_cont) < tol:
            print(f"\nConverged in {iteration + 1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge for continuous load within the maximum number of iterations = {iteration + 1}.")

    # Newton-Raphson iteration for point load
    print("\n--- Solving for Point Load ---")
    for iteration in range(max_iter):
        R_point = residual(v_point, M_point)
        print(f"Iteration {iteration + 1}: Residual Norm = {np.linalg.norm(R_point):.4e}")
        if np.linalg.norm(R_point) < tol:
            print(f"\nConverged in {iteration + 1} iterations.\n")
            break

        J_point = jacobian(v_point, M_point)
        try:
            # Solve the linear system J * delta_v = -R
            delta_v_point = np.linalg.solve(J_point, -R_point)
        except np.linalg.LinAlgError:
            # If Jacobian is singular, use pseudo-inverse for a least-squares solution
            print(f"\nJacobian is singular for iteration {iteration + 1}. Using pseudo-inverse.")
            delta_v_point = np.linalg.lstsq(J_point, -R_point, rcond=None)[0]  # Uses pseudo-inverse

        v_point += w * delta_v_point  # Update solution with relaxation factor

        if np.linalg.norm(delta_v_point) < tol:
            print(f"\nConverged in {iteration + 1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge for point load within the maximum number of iterations = {iteration + 1}.")

    # Prepare full solution including boundary conditions
    v_full_cont = np.zeros(N + 1)
    v_full_cont[:] = v_cont
    v_full_cont[0] = 0  # Ensure v(0) = 0 is strictly maintained

    v_full_point = np.zeros(N + 1)
    v_full_point[:] = v_point
    v_full_point[0] = 0  # Ensure v(0) = 0 is strictly maintained

    return x, v_full_point, v_full_cont, L, P, W, EI


# --- Main Execution and Visualization ---
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

'''
Important Considerations:
- Minimum value of 'N' can be 3.
- Do not take N > 350, as it may lead to the Jacobian Matrix becoming singular for all three kinds of initial guesses (random, uniform, zeros), hindering convergence.
'''
N = 250  # Number of grid spaces for discretization

# Get user input for initial guess and boundary condition method
init_guess = input("Enter initial guess type (random, uniform, zeros): ").strip().lower()
dv_dx_at_firstPoint = input("Enter dv/dx at x = 0 (FD2, FD1, Direct): ").strip().upper()

# Solve the nonlinear system using the Newton-Raphson method
x, v_point, v_cont, L, P, W, EI = newton_raphson_system(N, init_guess, dv_dx_at_firstPoint)

print("\n--- Generating Plots ---")

# Create a Pandas DataFrame to store and organize the results
df = pd.DataFrame({
    'x': x,
    'Newton-Raphson Solution (point load)': v_point,
    'Newton-Raphson Solution (continuous load)': v_cont,
    # Analytical solution for a cantilever beam with a point load at the free end
    'Euler-Bernoulli Solution (point load)': P * x ** 2 / (6 * EI) * (3 * L - x),
    # Analytical solution for a cantilever beam with a continuous load
    'Euler-Bernoulli Solution (continuous load)': -(W * L * x ** 3) / (6 * EI) + (W * x ** 4) / (24 * EI) + (W * (L ** 2) * x ** 2) / (4 * EI)
})

# Create an interactive plot using Plotly to visualize the beam deflections
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['x'], y=df['Newton-Raphson Solution (point load)'], mode='lines', name='Newton-Raphson Point Load'))
fig.add_trace(go.Scatter(x=df['x'], y=df['Newton-Raphson Solution (continuous load)'], mode='lines', name='Newton-Raphson Continuous Load'))
fig.add_trace(go.Scatter(x=df['x'], y=df['Euler-Bernoulli Solution (point load)'], mode='lines', name='Euler-Bernoulli Point Load', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=df['x'], y=df['Euler-Bernoulli Solution (continuous load)'], mode='lines', name='Euler-Bernoulli Continuous Load', line=dict(dash='dash')))

# Update plot layout for better readability
fig.update_layout(
    title='Beam Deflection Solutions',
    xaxis_title='Length along beam, x (m)',
    yaxis_title='Deflection, v(x) (m)',
    legend_title='Solution Type',
    hovermode='x unified' # Provides combined hover information across all traces
)

# Display the interactive plot
fig.show()

---
Implementation of Point Load and Uniform Continuous Load for N Number of Loads
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess, dv_dx_at_firstPoint, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.
        FD1: Finite Difference Method 1
        FD2: Finite Difference Method 2

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']
    valid_dv_dx_methods = ['FD2', 'FD1', 'Direct']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")
    if dv_dx_at_firstPoint not in valid_dv_dx_methods:
        raise ValueError(f"Invalid dv/dx method '{dv_dx_at_firstPoint}'. Must be one of {valid_dv_dx_methods}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the free end (N)
    W  = -100.0      # Continuous Load (N/m)
    w  =  0.5        # Relaxation factor

    # Initial guess
    v = np.zeros(N+1)          # Good guess. Achieves Middle Ground.
    v[0] = 0

    # Getting the Point Loads
    def get_point_loads():
        n = int(input("Enter the number of point loads: "))
        point_loads = []
        for i in range(n):
            pos, mag = map(float, input(f"Enter position and magnitude of point load {i+1} (space-separated): ").split())
            point_loads.append((pos, mag))
        return point_loads

    # Getting the Uniform Continuous Loads
    def get_continuous_loads():
        m = int(input("Enter the number of continuous loads: "))
        continuous_loads = []
        for i in range(m):
            start, end, intensity = map(float, input(f"Enter start position, end position, and intensity of continuous load {i+1} (space-separated): ").split())
            continuous_loads.append((start, end, intensity))
        return continuous_loads

    # Generating the Moment Equation
    def moment_equation(L, point_loads, continuous_loads, N):
        x_vals = np.linspace(0, L, N+1)
        moment_values = np.zeros(N+1)

        for i, x in enumerate(x_vals):
            M = 0
            # Calculate moment due to point loads
            for pos, mag in point_loads:
                if not (0 <= pos <= L):
                    raise ValueError(f"Point load at {pos} exceeds beam length {L}.")
                if x <= pos:
                    M += mag * (x - pos)

            # Calculate moment due to continuous loads
            for start, end, intensity in continuous_loads:
                if not (0 <= start <= end <= L):
                    raise ValueError(f"Continuous load from {start} to {end} exceeds beam length {L}.")

                if x <= start:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x
                elif start < x < end:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x - intensity * (x - start)**2 / 2
                else:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x - intensity * (end - start) * (x - (start + end) / 2)

            moment_values[i] = M

        return moment_values

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0          # Boundary condition v(0) = 0 (Deflection at fixed end is zero)

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0 (Slope at fixed end is zero)
                if dv_dx_at_firstPoint == 'FD2':
                    R[i] = (-3*v_full[i]+4*v_full[i+1]-v_full[i+2])/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    R[i] = (v_full[i+1]-v_full[i])/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    # This case seems inconsistent with the dv/dx = 0 boundary condition for i=0.
                    # It looks like a second derivative term.
                    R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2 + (M[i]/EI)
            elif i == N:
                # Equation for the last point (Free end boundary condition: d^2v/dx^2 = 0)
                # This formulation incorporates the non-linear term for large deflections.
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                # General equation for interior points
                # This formulation incorporates the non-linear term for large deflections.
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0          # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Derivatives of the boundary condition R[0] with respect to v_j
                if dv_dx_at_firstPoint == 'FD2':
                    # dR[0]/dv[0], dR[0]/dv[1], dR[0]/dv[2]
                    J[i, i], J[i, i+1], J[i, i+2] =  0, 4/(2*dx), -1/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    # dR[0]/dv[0], dR[0]/dv[1]
                    J[i, i], J[i, i+1] = 0, 1/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    # This derivative seems inconsistent with the dv/dx = 0 boundary condition for i=0.
                    # It looks like the derivative of a second derivative term.
                    J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2

            elif i == N:
                # Derivatives of R[N] with respect to v_j for the last point
                # J[N, N-3]
                J[i, i-3] = -1/dx**2
                # J[N, N-2]
                J[i, i-2] =  4/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (1)
                # J[N, N-1]
                J[i, i-1] = -5/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                # J[N, N]
                J[i, i]   =  2/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (3)
            else:
                # Derivatives of R[i] with respect to v_j for interior points
                if i == 1:
                    # Special handling for i=1 because v[0] is fixed to 0
                    J[i, i-1] = 0
                else:
                    # J[i, i-1]
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                # J[i, i]
                J[i, i]   = -2/dx**2
                # J[i, i+1]
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 +  1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Get user input for point and continuous loads
    point_loads = get_point_loads()
    continuous_loads = get_continuous_loads()
    # Calculate moment values based on user-defined loads
    moment_values = moment_equation(L, point_loads, continuous_loads, N)
    print()

    # Newton-Raphson iteration loop
    for iteration in range(max_iter):
        # Compute the residual vector
        R = residual(v, moment_values)
        print(f"Residual = {np.linalg.norm(R)}")
        # Check for convergence based on residual norm
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        # Compute the Jacobian matrix
        J = jacobian(v, moment_values)
        try:
            # Attempt to solve the linear system J * delta_v = -R for delta_v
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If Jacobian is singular, use pseudo-inverse (least squares)
            print(f"Jacobian is singular for iteration {iteration+1}. Using pseudo-inverse.\n")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        # Update the solution vector with relaxation factor 'w'
        v += w*delta_v

        # Check for convergence based on the change in solution vector
        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        # Raise an error if convergence is not achieved within max_iter
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Create the full solution array, ensuring the first element is 0
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0

    return x, v_full, L, P, W, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.   >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250
# Get user input for initial guess type and dv/dx method
init_guess = input("Enter initial guess type (random, uniform, zeros): ")
dv_dx_at_firstPoint = input("Enter dv/dx at x = 0 (FD2, FD1, Direct): ")
# Call the Newton-Raphson system solver
x, v, L, P, W, EI = newton_raphson_system(N, init_guess, dv_dx_at_firstPoint)
# print(v)
print()

# Create Pandas DataFrame to store results
df = pd.DataFrame({
    'x': x,
    'Newton-Raphson Solution': v,
    'Euler-Bernoulli (Point Load)': P*x**2/(6*EI) * (3*L - x),
    'Euler-Bernoulli (Continuous Load)': -(W*L*x**3)/(6*EI) + (W*x**4)/(24*EI) + (W*(L**2)*x**2)/(4*EI)
})

# Create plot using Plotly
fig = go.Figure()

# First trace: Newton-Raphson Solution
fig.add_trace(go.Scatter(
    x=df['x'],
    y=df['Newton-Raphson Solution'],
    name='Newton-Raphson',
    line=dict(color='blue'))
)

# Second trace: Euler-Bernoulli solution for point load
fig.add_trace(go.Scatter(
    x=df['x'],
    y=df['Euler-Bernoulli (Point Load)'],
    name='Euler-Bernoulli (Point)',
    line=dict(color='orange', dash='dash'))
)

# Third trace: Euler-Bernoulli solution for continuous load
fig.add_trace(go.Scatter(
    x=df['x'],
    y=df['Euler-Bernoulli (Continuous Load)'],
    name='Euler-Bernoulli (Continuous)',
    line=dict(color='green', dash='dot'))
)

# Update plot layout for titles and labels
fig.update_layout(
    title='Beam Deflection Solutions',
    xaxis_title='Position (x)',
    yaxis_title='Deflection (v)',
    legend=dict(title='Solution Type'),
    template='plotly_white'
)

# Display the plot
fig.show()